In [3]:
import pandas as pd
from konlpy.tag import Mecab

df = pd.read_csv('csv/jobkorea_question_labeled_mecab.csv')

# 한국어 불용어 불러오기
# 공백문자 제거하고 set으로 저장함(중복제거)
with open('py/stopwords-ko.txt', 'r', encoding='utf-8') as f:
    stopwords = set(line.strip() for line in f)

# 텍스트를 입력받아 불용어 처리 후 Mecab을 사용하여 품사 태깅을 수행하는 함수
def mecab_tokenize(text, stopwords):
    words = []
    mecab = Mecab(dicpath=r"C:/mecab/mecab-ko-dic")
    tokens = mecab.morphs(text)
    # 불용어가 아닌 단어들을 태깅
    filtered_tokens = [token for token in tokens if token not in stopwords]
    for token in filtered_tokens:
        tagged_token = mecab.pos(token)
        # 품사 태그를 제외하고 단어만 추출하여 리스트에 추가
        words += [word for word, tag in tagged_token]
    return words  # 태깅된 단어들

# 데이터프레임에 불용어 처리된 단어 추출 결과를 저장
df['pos'] = df['answer'].apply(lambda x: mecab_tokenize(x, stopwords))
df.to_csv('mecab_without_tag.csv')


In [ ]:
import pandas as pd
from konlpy.tag import Okt

df = pd.read_csv('csv/jobkorea_unique0.95.csv')

# 한국어 불용어 불러오기
# 공백문자 제거하고 set으로 저장함(중복제거)
with open('py/stopwords-ko.txt', 'r', encoding='utf-8') as f:
    stopwords = set(line.strip() for line in f)

# 텍스트를 입력받아 불용어 처리 후 Okt를 사용하여 품사 태깅을 수행하는 함수
def okt_tokenize(text, stopwords):
    words = []
    okt = Okt()
    tokens = okt.morphs(text)
    filtered_tokens = [token for token in tokens if token not in stopwords]
    for token in filtered_tokens:
        tagged_token = okt.pos(token)
        words += [word for word, tag in tagged_token]
    return words


# 데이터프레임에 불용어 처리된 품사 태깅 결과를 저장
df['pos'] = df['answer'].apply(lambda x: okt_tokenize(x, stopwords))
df.to_csv('jobkorea_pos_okt_NV.csv')


In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# 데이터 불러오기
df = pd.read_csv('mecab_without_tag.csv')

# 기업별로 자기소개 합치기
grouped_df = df.groupby('company')['pos'].apply(' '.join).reset_index()


# TfidfVectorizer 객체 생성
vectorizer = TfidfVectorizer(max_features=10)  # 상위 10개의 중요 키워드를 추출하려면 max_features를 10으로 설정

# 기업별 자기소개에서 중요 키워드 추출
tfidf_matrix = vectorizer.fit_transform(grouped_df['pos'])

# 추출된 키워드 DataFrame에 저장
keywords_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

# 기업별 중요 키워드 출력
for idx, company in grouped_df.iterrows():
    print(f"{company['company']}의 중요 키워드:")
    print(keywords_df.loc[idx].sort_values(ascending=False))
    print("\n")


(사)대한산업안전협회의 중요 키워드:
고객     0.631193
위해     0.565731
경험     0.449439
통해     0.225576
사람     0.125066
대한     0.060290
합니다    0.056609
입니다    0.055931
생각     0.055262
업무     0.000000
Name: 0, dtype: float64


(사)대한체육회의 중요 키워드:
입니다    0.531571
위해     0.477931
통해     0.476418
업무     0.309723
대한     0.254667
경험     0.237304
생각     0.175070
합니다    0.119559
고객     0.000000
사람     0.000000
Name: 1, dtype: float64


(사)한국정보산업연합회의 중요 키워드:
업무     0.585558
입니다    0.472426
통해     0.346426
생각     0.297038
사람     0.288104
경험     0.258833
합니다    0.217342
위해     0.130322
대한     0.092590
고객     0.053853
Name: 2, dtype: float64


(사)한국지방재정공제회의 중요 키워드:
통해     0.757816
경험     0.328234
업무     0.274177
생각     0.258297
합니다    0.231519
위해     0.198319
사람     0.182676
대한     0.176125
입니다    0.130712
고객     0.040975
Name: 3, dtype: float64


(사)한국화재보험협회의 중요 키워드:
경험     0.604466
업무     0.450819
입니다    0.429850
생각     0.424707
대한     0.185341
사람     0.096118
합니다    0.087012
위해     0.086957
통해     0.086681
고객     0

In [7]:
import pandas as pd
from konlpy.tag import Mecab
from sklearn.feature_extraction.text import TfidfVectorizer

df = pd.read_csv('csv/jobkorea_question_labeled_mecab.csv')

# 한국어 불용어 불러오기
# 공백문자 제거하고 set으로 저장함(중복제거)
with open('py/stopwords-ko.txt', 'r', encoding='utf-8') as f:
    stopwords = set(line.strip() for line in f)

# 텍스트를 입력받아 불용어 처리 후 Mecab을 사용하여 품사 태깅을 수행하는 함수
def mecab_tokenize(text, stopwords):
    Tag = []
    mecab = Mecab(dicpath=r"C:/mecab/mecab-ko-dic")
    tokens = mecab.morphs(text)
    # 불용어가 아닌 단어들을 태깅
    filtered_tokens = [token for token in tokens if token not in stopwords]
    for token in filtered_tokens:
        tagged_token = mecab.pos(token)
        # 'NNG' 태그를 가진 토큰만 추가
        # ('단어','태그')형태이므로 단어 부분만 참조
        Tag += [t[0] for t in tagged_token if t[1] in ('NNG')]
    return Tag  # 태깅된 토큰들의 단어만

# 데이터프레임에 불용어 처리된 품사 태깅 결과를 저장
df['pos1'] = df['answer'].apply(lambda x: mecab_tokenize(x, stopwords))

# 기업별로 자기소개 합치기
grouped_df = df.groupby('company')['pos1'].apply(lambda x: ' '.join(' '.join(pos) for pos in x)).reset_index()

# TfidfVectorizer 객체 생성
vectorizer = TfidfVectorizer(max_features=10)  # 상위 10개의 중요 키워드를 추출하려면 max_features를 10으로 설정

# 기업별 자기소개에서 중요 키워드 추출
tfidf_matrix = vectorizer.fit_transform(grouped_df['pos1'])


# 추출된 키워드 DataFrame에 저장
keywords_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

# 기업별 중요 키워드 출력
for idx, company in grouped_df.iterrows():
    print(f"{company['company']}의 중요 키워드:")
    print(keywords_df.loc[idx].sort_values(ascending=False))
    print("\n")


KeyError: 'pos'